In [4]:
#notebook to download the csv of edges and nodes of a given network
import os
import requests 
import time
import string
import networkx as nx
import itertools
import networkx as nx
import pandas as pd
import json
import re
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords

from nltk.corpus import wordnet as wn #importing it
from nltk.stem.wordnet import WordNetLemmatizer #importing wordnet lemmatizer
from nltk import pos_tag #part-of-speech-tagger
from collections import defaultdict #defaultdict returns default value for non-existant keys you try to  access based on the function you passed in the constructor
from google.colab import files

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
def extract_text(df):       #extract the text from the tweets and RT
                            #works ONLY on .csv file
  list_strings = []
  for index in range(len(df)):
    if index % 1000 == 0:
      print(str(index)+' / '+str(len(df)))
    text = df.loc[index]['text']                          #if it is nor trucated nor a RT  i take "text"
    string = -1
    if (df.loc[index,"truncated"] == True):                 #if it is trucated I take "extended_tweet"
        string = df.loc[index,"extended_tweet"]
    if type(df.loc[index,"retweeted_status"]) != float:     #if it is a RT I take retweeted_status
        string = df.loc[index,"retweeted_status"]
    if type(string) == str :
        if(re.search('full_text\':(.+?)https',string) != None):     #if I find "full_text"
          s = re.search('full_text\':(.+?)https',string).group(1)
        if(re.search('text\':(.+?)https',string)!= None):
          s = re.search('text\':(.+?)https',string).group(1)
        else: 
          continue
        list_strings.append(s)
        #print(s)         
    else:
      list_strings.append(text)
      #print(text)
      

  return list_strings

In [14]:
# Cleaning, lemmatising and pos tagging tweets

nltk.download('words')
WORDS = set(nltk.corpus.words.words()) #the last two lines serve to download the corpus of standard English language words
nltk.download('stopwords') #downloading stopwords
STOP_WORDS = set(nltk.corpus.stopwords.words("english")) #taking the stop words from English language
nltk.download('wordnet') #downloading wordnet
nltk.download('averaged_perceptron_tagger') #downloading tagger
tag_map = defaultdict(lambda : wn.NOUN) #here we define that wn.NOUN is the default value for the dict
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

def lemma_pos_cleaner(tweet):

    tweet = re.sub("@[A-Za-z0-9]+","",tweet) # remove mentions
    tweet = re.sub("#[A-Za-z0-9]+", "",tweet) # remove hashtags
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) # remove http links
    tweet = " ".join(tweet.split())
    tweet = str.lower(tweet) #to lowercase 
    tweet = re.sub("'"," ",tweet) # remove aphostrophe

    #basically we use pos_tag function on tokens that we get by applying wordpunct tokenization
    #to tweet (it separates all the words and symbols)
    #then we pass the token along with it's wordnet pos value that we get from the tag_map dictionary (noun, adjective, verb or adverb) to the lemma function (the WordNetLemmatizer())
    lemma_function = WordNetLemmatizer()
    tweet = " ".join(lemma_function.lemmatize(token, tag_map[tag[0]]) for token, tag in nltk.pos_tag(nltk.wordpunct_tokenize(tweet))) #lemmatize
  

    # francesco: I removed also all 2 letters words and added specific words, words that appears frequently but are discarded because they are not in the english language
    SPECIFIC_WORDS = ['virus', 'coronavirus', 'covid19', 'covid', 'trump', 'hubei', 'beijing', 'xinjiang', 'jinping', 'korea', 'xinhua', 'india', 'taiwan','johnson','singapore', 'africa', 'japanese', 'france', 'asian', 'australia', 'french', 'asia', 'leishenshan', 'british', 'qingdao', 'fauci', 'america',  'california', 'sichuan', 'malaysia', 'huawei','thailand', 'shandong', 'italy', 'philippines', 'germany', 'facebook', 'african', 'shenzhen', 'tokyo', 'russian','uygur', '5g', 'pompeo', 'vietnam', 'australian', 'cambodia', 'zhejiang', 'yunnan', 'guangdong', 'korean', 'iran', 'washington']
    tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) if (w in WORDS or w in SPECIFIC_WORDS) and len(w)>2 and w not in STOP_WORDS ) #remove stop words
   
    return tweet

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [7]:
def frequency_dictionary(df):
  unique_words = {}

  for row in df:
    for word in row.split():
      #if the word is encountered for the first time add to dict as key and set its value to 0
      unique_words.setdefault(word,0)
      #increase the value (i.e the count) of the word by 1 every time it is encountered
      unique_words[word] += 1

  return unique_words

In [179]:
period = '_SeptOct2020'  # _JanFeb2020, _MarchApril2021, _SeptOct2020

In [180]:
China = pd.read_csv('/content/China'+period+'.csv')
USA = pd.read_csv('/content/USA'+period+'.csv')
China_USA = pd.read_csv('/content/China&USA'+period+'.csv')

In [181]:
# number of tweets:
print('China: ', len(China))
print('USA: ', len(USA))
print('China&USA: ', len(China_USA))

China:  3098
USA:  8389
China&USA:  11487


In [182]:
text_China = extract_text(China)
text_USA = extract_text(USA)
text_China_USA = extract_text(China_USA)

0 / 3098
1000 / 3098
2000 / 3098
3000 / 3098
0 / 8389
1000 / 8389
2000 / 8389
3000 / 8389
4000 / 8389
5000 / 8389
6000 / 8389
7000 / 8389
8000 / 8389
0 / 11487
1000 / 11487
2000 / 11487
3000 / 11487
4000 / 11487
5000 / 11487
6000 / 11487
7000 / 11487
8000 / 11487
9000 / 11487
10000 / 11487
11000 / 11487


In [183]:
cleaned_text_China = [lemma_pos_cleaner(txt) for txt in text_China]
cleaned_text_USA = [lemma_pos_cleaner(txt) for txt in text_USA]
cleaned_text_China_USA = [lemma_pos_cleaner(txt) for txt in text_China_USA]

print('China:')
print(cleaned_text_China[0:10])
print()
print('USA:')
print(cleaned_text_USA[0:10])
print()
print('China&USA:')
print(cleaned_text_China_USA[0:10])

China:
['people beijing china tour popular destination like forbidden city pose wed picture shop car enjoy visit fall weather', 'erect people volunteer martyr cemetery northeastern china begin', 'china international medical equipment fair become showcase state art medical', 'new united nation report show china experience positive growth third quarter despite shrink', 'detect cold chain food east shandong', 'five year review china commit green development', 'report china country see positive trade growth', 'revenue first three quarter', 'potential record breaker tall male reach meter seven foot three inch age junior high student china sichuan furniture clothes distinguished trait', 'china build hospital within day infrastructure development one key point address country high speed rail new airport greatly develop past five year']

USA:
['india china country claim part region history', 'china ask india pledge return apprehend soldier', 'australia join india japan united state large scale

In [184]:
freq_dict_China = frequency_dictionary(cleaned_text_China)
freq_dict_China = dict(sorted(freq_dict_China.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_USA = frequency_dictionary(cleaned_text_USA)
freq_dict_USA = dict(sorted(freq_dict_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_China_USA = frequency_dictionary(cleaned_text_China_USA)
freq_dict_China_USA = dict(sorted(freq_dict_China_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

# number of words in the cleaned tweets:
print('China: ', len(list(freq_dict_China)))
print('USA: ', len(list(freq_dict_USA)))
print('China&USA: ', len(list(freq_dict_China_USA)))

China:  4216
USA:  4967
China&USA:  6382


In [185]:
# Most frequent words
print('China')
print([key for key in freq_dict_China.keys() if freq_dict_China[key]>200])
print()
print('USA')
print([key for key in freq_dict_USA.keys() if freq_dict_USA[key]>400])
print()
print('China&USA')
print([key for key in freq_dict_China_USA.keys() if freq_dict_China_USA[key]>600])

China
['china', 'covid', 'say', 'case', 'new', 'coronavirus', 'country', 'year', 'day', 'president', 'report', 'city', 'national', 'people', 'world']

USA
['covid', 'coronavirus', 'say', 'trump', 'new', 'vaccine', 'china', 'case', 'president', 'test', 'pandemic', 'report', 'positive', 'house', 'day', 'people', 'state']

China&USA
['covid', 'china', 'coronavirus', 'say', 'new', 'trump', 'case', 'vaccine', 'president', 'test', 'report', 'pandemic', 'day', 'positive', 'people']


# less frequent words

In [186]:
# less frequent words:
print('Less frequent China: ', len([key for key in freq_dict_China.keys() if freq_dict_China[key]<5]))
print('More frequent China: ', len([key for key in freq_dict_China.keys() if freq_dict_China[key]>=5]))
less_frequent_words_China = [key for key in freq_dict_China.keys() if freq_dict_China[key]<5]

print('Less frequent USA: ', len([key for key in freq_dict_USA.keys() if freq_dict_USA[key]<5]))
print('More frequent USA:', len([key for key in freq_dict_USA.keys() if freq_dict_USA[key]>=5]))
less_frequent_words_USA = [key for key in freq_dict_USA.keys() if freq_dict_USA[key]<5]

print('Less frequent China&USA: ', len([key for key in freq_dict_China_USA.keys() if freq_dict_China_USA[key]<5]))
print('More frequent China&USA: ', len([key for key in freq_dict_China_USA.keys() if freq_dict_China_USA[key]>=5]))
less_frequent_words_China_USA = [key for key in freq_dict_China_USA.keys() if freq_dict_China_USA[key]<5]

Less frequent China:  2825
More frequent China:  1391
Less frequent USA:  2759
More frequent USA: 2208
Less frequent China&USA:  3571
More frequent China&USA:  2811


In [187]:
# discard less frequent words
cleaned_mostfreq_text_China = cleaned_text_China.copy()
for txt in range(len(cleaned_mostfreq_text_China)):
  if txt % 1000 == 0:
    print(txt, '/',len(cleaned_mostfreq_text_China))
  for word in less_frequent_words_China:
    if word in cleaned_mostfreq_text_China[txt].split():
      cleaned_mostfreq_text_China[txt] = cleaned_mostfreq_text_China[txt].replace(word, '')
      cleaned_mostfreq_text_China[txt] = " ".join(cleaned_mostfreq_text_China[txt].split())

cleaned_mostfreq_text_USA = cleaned_text_USA.copy()
for txt in range(len(cleaned_mostfreq_text_USA)):
  if txt % 1000 == 0:
    print(txt, '/',len(cleaned_mostfreq_text_USA))
  for word in less_frequent_words_USA:
    if word in cleaned_mostfreq_text_USA[txt].split():
      cleaned_mostfreq_text_USA[txt] = cleaned_mostfreq_text_USA[txt].replace(word, '')
      cleaned_mostfreq_text_USA[txt] = " ".join(cleaned_mostfreq_text_USA[txt].split())

cleaned_mostfreq_text_China_USA = cleaned_text_China_USA.copy()
for txt in range(len(cleaned_mostfreq_text_China_USA)):
  if txt % 1000 == 0:
    print(txt, '/',len(cleaned_mostfreq_text_China_USA))
  for word in less_frequent_words_China_USA:
    if word in cleaned_mostfreq_text_China_USA[txt].split():
      cleaned_mostfreq_text_China_USA[txt] = cleaned_mostfreq_text_China_USA[txt].replace(word, '')
      cleaned_mostfreq_text_China_USA[txt] = " ".join(cleaned_mostfreq_text_China_USA[txt].split())

0 / 3089
1000 / 3089
2000 / 3089
3000 / 3089
0 / 8384
1000 / 8384
2000 / 8384
3000 / 8384
4000 / 8384
5000 / 8384
6000 / 8384
7000 / 8384
8000 / 8384
0 / 11473
1000 / 11473
2000 / 11473
3000 / 11473
4000 / 11473
5000 / 11473
6000 / 11473
7000 / 11473
8000 / 11473
9000 / 11473
10000 / 11473
11000 / 11473


In [188]:
freq_dict_China = frequency_dictionary(cleaned_mostfreq_text_China)
freq_dict_China = dict(sorted(freq_dict_China.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_USA = frequency_dictionary(cleaned_mostfreq_text_USA)
freq_dict_USA = dict(sorted(freq_dict_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_China_USA = frequency_dictionary(cleaned_mostfreq_text_China_USA)
freq_dict_China_USA = dict(sorted(freq_dict_China_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

# number of words in the cleaned tweets:
print('China: ', len(list(freq_dict_China)))
print('USA: ', len(list(freq_dict_USA)))
print('China&USA: ', len(list(freq_dict_China_USA)))

China:  1408
USA:  2222
China&USA:  2828


# Build Network

In [189]:
df_China = pd.DataFrame.from_dict(freq_dict_China, orient='index').reset_index()
df_China.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China.sort_values(by=['Count'], ascending=False, inplace=True)
df_China.reset_index(inplace=True)
df_China.drop(columns="index",inplace=True)

df_USA = pd.DataFrame.from_dict(freq_dict_USA, orient='index').reset_index()
df_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_USA.reset_index(inplace=True)
df_USA.drop(columns="index",inplace=True)

df_China_USA = pd.DataFrame.from_dict(freq_dict_China_USA, orient='index').reset_index()
df_China_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_China_USA.reset_index(inplace=True)
df_China_USA.drop(columns="index",inplace=True)

print('China')
print(df_China.iloc[0:30])
print()
print('USA')
print(df_USA.iloc[0:30])
print()
print('China&USA')
print(df_China_USA.iloc[0:30])
print()

China
           Word  Count
0         china   1751
1         covid    641
2           say    446
3          case    324
4           new    321
5   coronavirus    272
6       country    257
7          year    256
8           day    256
9     president    238
10       report    231
11         city    227
12     national    216
13       people    211
14        world    204
15         test    193
16     province    192
17     pandemic    189
18       health    171
19        first    167
20      million    167
21        watch    154
22       global    153
23      vaccine    148
24         live    134
25        trump    129
26         take    127
27          see    124
28     positive    122
29         high    121

USA
           Word  Count
0         covid   4188
1   coronavirus   2205
2           say   1662
3         trump   1315
4           new   1170
5       vaccine   1168
6         china   1073
7          case   1055
8     president   1017
9          test   1006
10     pandemic    602


In [190]:
keys_China = freq_dict_China.keys()  
keys_USA = freq_dict_USA.keys()  
keys_China_USA = freq_dict_China_USA.keys()  

In [151]:
def create_network(cleaned_text):
  network = {}
  #connect the word that appear in the same tweets
  for row in cleaned_text:
    combined_list = [word for word in str.split(row)]
    #for pair in itertools.product(combined_list, combined_list):
    #print(combined_list)
    for pair in itertools.product(combined_list, combined_list):
          #exclude self-loops and count each pair only once because our graph is undirected and we do not take self-loops into account
          if pair[0]!=pair[1] and not(pair[::-1] in network):
              network.setdefault(pair,0)
              network[pair] += 1 
  network_df = pd.DataFrame.from_dict(network, orient="index")
  network_df.columns = ["weight"]
  network_df.sort_values(by="weight",inplace=True, ascending=False)
  return network, network_df

In [191]:
network_China, network_df_China = create_network(cleaned_mostfreq_text_China)
network_USA, network_df_USA = create_network(cleaned_mostfreq_text_USA)
network_China_USA, network_df_China_USA = create_network(cleaned_mostfreq_text_China_USA)

In [192]:
print('China:')
print(network_df_China.iloc[0:30])
print()
print('USA:')
print(network_df_USA.iloc[0:30])
print()
print('China&USA:')
print(network_df_China_USA.iloc[0:30])
print()

China:
                      weight
(china, say)             283
(year, china)            218
(china, covid)           212
(china, day)             211
(china, national)        185
(report, case)           183
(new, case)              181
(national, day)          170
(china, country)         152
(covid, case)            152
(china, province)        151
(china, city)            149
(poverty, china)         127
(covid, say)             125
(new, china)             125
(new, report)            124
(people, china)          121
(day, holiday)           120
(china, world)           120
(test, positive)         117
(new, covid)             117
(china, xinjiang)        111
(president, china)       111
(china, holiday)         108
(covid, pandemic)        108
(china, development)     108
(first, china)           105
(global, china)          104
(say, president)         101
(health, case)            97

USA:
                          weight
(president, trump)          1037
(say, covid)          

#Graph


In [193]:
def get_graph(network):
  up_weighted = []
  for edge in network:
      #we can filter edges by weight by uncommenting the next line and setting desired weight threshold
      up_weighted.append((edge[0],edge[1],network[edge]))
      
  #print(network)
  #print(up_weighted[0:10])
  G = nx.Graph()
  G.add_weighted_edges_from(up_weighted)
  return G

In [194]:
G_China = get_graph(network_China)
G_USA = get_graph(network_USA)
G_China_USA = get_graph(network_China_USA)

In [195]:
print('China:')
print('Nodes: ',len(G_China.nodes()))
print('Edges: ',len(G_China.edges()))
print('Is connected: ',nx.is_connected(G_China))
print()
print('USA:')
print('Nodes: ',len(G_USA.nodes()))
print('Edges: ',len(G_USA.edges()))
print('Is connected: ',nx.is_connected(G_USA))
print()
print('China&USA:')
print('Nodes: ',len(G_China_USA.nodes()))
print('Edges: ',len(G_China_USA.edges()))
print('Is connected: ',nx.is_connected(G_China_USA))

China:
Nodes:  1407
Edges:  80335
Is connected:  True

USA:
Nodes:  2222
Edges:  148901
Is connected:  True

China&USA:
Nodes:  2828
Edges:  232823
Is connected:  True


# PageRank

In [196]:
# Calculating the pagerank on graph G, teleportation probability here is 0.15 but since the graph is strongly connected we can set it to zero if we want
pr_China = nx.algorithms.pagerank(G_China,alpha = 1)
pr_China = dict(sorted(pr_China.items(), key=lambda item: item[1],reverse  = True))

pr_USA = nx.algorithms.pagerank(G_USA,alpha = 1)
pr_USA = dict(sorted(pr_USA.items(), key=lambda item: item[1],reverse  = True))

pr_China_USA = nx.algorithms.pagerank(G_China_USA,alpha = 1)
pr_China_USA = dict(sorted(pr_China_USA.items(), key=lambda item: item[1],reverse  = True))

In [197]:
def threshold(vector,threshold):

  l = [(el,vector[el]) for el in vector if vector[el] >= threshold ]

  return pd.DataFrame(l)

In [159]:
def threshold_reverse(vector,threshold):

  l = [(el,vector[el]) for el in vector if vector[el] < threshold ]

  return pd.DataFrame(l)

In [198]:
print('China: ', len(threshold(pr_China,0.0003)))
print()
print('first:')
print(threshold(pr_China,0.0003).iloc[:30])
print()
print('last:')
print(threshold(pr_China,0.0003).iloc[len(threshold(pr_China,0.0003))-30:])
print()
print('USA: ', len(threshold(pr_USA,0.0003)))
print()
print('first:')
print(threshold(pr_USA,0.0003).iloc[:30])
print()
print('last:')
print(threshold(pr_USA,0.0003).iloc[len(threshold(pr_USA,0.0003))-30:])
print()
print('China&USA: ', len(threshold(pr_China_USA,0.0003)))
print()
print('first:')
print(threshold(pr_China_USA,0.0003).iloc[:30])
print()
print('last:')
print(threshold(pr_China_USA,0.0003).iloc[len(threshold(pr_China_USA,0.0003))-30:])

China:  794

first:
              0         1
0         china  0.051136
1         covid  0.018314
2           say  0.014814
3           new  0.010169
4          case  0.009960
5           day  0.009110
6          year  0.008864
7       country  0.008656
8          city  0.007800
9        report  0.007610
10  coronavirus  0.007489
11    president  0.007483
12     national  0.007045
13        world  0.006720
14       people  0.006641
15     province  0.006114
16         test  0.005847
17       health  0.005752
18     pandemic  0.005589
19      million  0.005586
20        first  0.005532
21       global  0.004785
22      holiday  0.004423
23       accord  0.004075
24         high  0.004054
25      vaccine  0.004019
26         take  0.003964
27  development  0.003921
28      poverty  0.003904
29          see  0.003827

last:
               0         1
764       garden  0.000312
765         even  0.000312
766         wall  0.000312
767       person  0.000312
768        cover  0.000312
769  

# TF-IDF

In [61]:
tfidf = TfidfVectorizer(ngram_range=(1,1))   # ngram range can be changed to obtain measures regarding n grams instead of single words

X_China = tfidf.fit_transform(cleaned_mostfreq_text_China).toarray()    # entry (i,j) if Tfidf measure of word_list[j] in document i
word_list_China = tfidf.get_feature_names_out()

X_USA = tfidf.fit_transform(cleaned_mostfreq_text_USA).toarray()
word_list_USA = tfidf.get_feature_names_out()

X_China_USA = tfidf.fit_transform(cleaned_mostfreq_text_China_USA).toarray()
word_list_China_USA = tfidf.get_feature_names_out()


In [62]:
tfidf_df_China = pd.DataFrame(X_China,columns = word_list_China)

tfidf_df_USA = pd.DataFrame(X_USA,columns = word_list_USA)

tfidf_df_China_USA = pd.DataFrame(X_China_USA,columns = word_list_China_USA)

In [63]:
tfidf_word_measure_China = np.mean(tfidf_df_China,axis = 0)
tfidf_word_measure_China = tfidf_word_measure_China.sort_values(ascending = False)
tfidf_word_measure_USA = np.mean(tfidf_df_USA,axis = 0)
tfidf_word_measure_USA = tfidf_word_measure_USA.sort_values(ascending = False)
tfidf_word_measure_China_USA = np.mean(tfidf_df_China_USA,axis = 0)
tfidf_word_measure_China_USA = tfidf_word_measure_China_USA.sort_values(ascending = False)

In [64]:
print('China:')
print(tfidf_word_measure_China[0:30])
print()
print('USA:')
print(tfidf_word_measure_USA[0:30])
print()
print('China&USA:')
print(tfidf_word_measure_China_USA[0:30])
print()

China:
china          0.057393
covid          0.032344
case           0.029077
novel          0.027626
coronavirus    0.027115
new            0.025339
vaccine        0.022859
hospital       0.022712
say            0.020808
outbreak       0.019470
report         0.017295
fight          0.016957
patient        0.016629
people         0.015936
confirm        0.014500
watch          0.014347
health         0.014288
country        0.014000
day            0.013577
live           0.013453
province       0.012905
first          0.012837
city           0.012793
death          0.012317
year           0.012240
world          0.011623
epidemic       0.011616
amid           0.011082
medical        0.011039
president      0.010959
dtype: float64

USA:
coronavirus    0.049972
covid          0.048809
china          0.041930
vaccine        0.031179
case           0.028555
say            0.027612
new            0.026551
report         0.017860
death          0.017719
outbreak       0.017477
virus       

# reduced graph

In [199]:
# less frequent words:
less_important_words_China = [key for key in list(threshold_reverse(pr_China,0.0003)[0])]

less_important_words_USA = [key for key in list(threshold_reverse(pr_USA,0.0003)[0])]

less_important_words_China_USA = [key for key in list(threshold_reverse(pr_China_USA,0.0003)[0])]

In [200]:
# discard less frequent words
cleaned_mostimp_text_China = cleaned_mostfreq_text_China.copy()
for txt in range(len(cleaned_mostimp_text_China)):
  if txt % 1000 == 0:
    print(txt, '/',len(cleaned_mostimp_text_China))
  for word in less_important_words_China:
    if word in cleaned_mostimp_text_China[txt].split():
      cleaned_mostimp_text_China[txt] = cleaned_mostimp_text_China[txt].replace(word, '')
      cleaned_mostimp_text_China[txt] = " ".join(cleaned_mostimp_text_China[txt].split())

cleaned_mostimp_text_USA = cleaned_mostfreq_text_USA.copy()
for txt in range(len(cleaned_mostimp_text_USA)):
  if txt % 1000 == 0:
    print(txt, '/',len(cleaned_mostimp_text_USA))
  for word in less_important_words_USA:
    if word in cleaned_mostimp_text_USA[txt].split():
      cleaned_mostimp_text_USA[txt] = cleaned_mostimp_text_USA[txt].replace(word, '')
      cleaned_mostimp_text_USA[txt] = " ".join(cleaned_mostimp_text_USA[txt].split())

cleaned_mostimp_text_China_USA = cleaned_mostfreq_text_China_USA.copy()
for txt in range(len(cleaned_mostimp_text_China_USA)):
  if txt % 1000 == 0:
    print(txt, '/',len(cleaned_mostimp_text_China_USA))
  for word in less_important_words_China_USA:
    if word in cleaned_mostimp_text_China_USA[txt].split():
      cleaned_mostimp_text_China_USA[txt] = cleaned_mostimp_text_China_USA[txt].replace(word, '')
      cleaned_mostimp_text_China_USA[txt] = " ".join(cleaned_mostimp_text_China_USA[txt].split())

0 / 3089
1000 / 3089
2000 / 3089
3000 / 3089
0 / 8384
1000 / 8384
2000 / 8384
3000 / 8384
4000 / 8384
5000 / 8384
6000 / 8384
7000 / 8384
8000 / 8384
0 / 11473
1000 / 11473
2000 / 11473
3000 / 11473
4000 / 11473
5000 / 11473
6000 / 11473
7000 / 11473
8000 / 11473
9000 / 11473
10000 / 11473
11000 / 11473


In [201]:
freq_dict_China = frequency_dictionary(cleaned_mostimp_text_China)
freq_dict_China = dict(sorted(freq_dict_China.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_USA = frequency_dictionary(cleaned_mostimp_text_USA)
freq_dict_USA = dict(sorted(freq_dict_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_China_USA = frequency_dictionary(cleaned_mostimp_text_China_USA)
freq_dict_China_USA = dict(sorted(freq_dict_China_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

# number of words in the cleaned tweets:
print('China: ', len(list(freq_dict_China)))
print('USA: ', len(list(freq_dict_USA)))
print('China&USA: ', len(list(freq_dict_China_USA)))

China:  804
USA:  720
China&USA:  771


In [202]:
df_China = pd.DataFrame.from_dict(freq_dict_China, orient='index').reset_index()
df_China.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China.sort_values(by=['Count'], ascending=False, inplace=True)
df_China.reset_index(inplace=True)
df_China.drop(columns="index",inplace=True)

df_USA = pd.DataFrame.from_dict(freq_dict_USA, orient='index').reset_index()
df_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_USA.reset_index(inplace=True)
df_USA.drop(columns="index",inplace=True)

df_China_USA = pd.DataFrame.from_dict(freq_dict_China_USA, orient='index').reset_index()
df_China_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_China_USA.reset_index(inplace=True)
df_China_USA.drop(columns="index",inplace=True)

print('China')
print(df_China.iloc[0:30])
print()
print('USA')
print(df_USA.iloc[0:30])
print()
print('China&USA')
print(df_China_USA.iloc[0:30])
print()

China
           Word  Count
0         china   1751
1         covid    641
2           say    446
3          case    324
4           new    321
5   coronavirus    272
6          year    256
7       country    256
8           day    256
9     president    238
10       report    231
11         city    227
12     national    216
13       people    211
14        world    204
15         test    193
16     province    192
17     pandemic    189
18       health    171
19        first    167
20      million    167
21        watch    154
22       global    153
23      vaccine    148
24         live    134
25        trump    129
26         take    127
27          see    124
28     positive    122
29         high    121

USA
           Word  Count
0         covid   4188
1   coronavirus   2203
2           say   1662
3         trump   1315
4           new   1170
5       vaccine   1167
6         china   1073
7          case   1055
8     president   1017
9          test   1006
10     pandemic    602


In [203]:
keys_China = freq_dict_China.keys()  
keys_USA = freq_dict_USA.keys()  
keys_China_USA = freq_dict_China_USA.keys()  

In [204]:
network_China, network_df_China = create_network(cleaned_mostimp_text_China)
network_USA, network_df_USA = create_network(cleaned_mostimp_text_USA)
network_China_USA, network_df_China_USA = create_network(cleaned_mostimp_text_China_USA)

In [205]:
print('China:')
print(network_df_China.iloc[0:30])
print()
print('USA:')
print(network_df_USA.iloc[0:30])
print()
print('China&USA:')
print(network_df_China_USA.iloc[0:30])
print()

China:
                      weight
(china, say)             283
(year, china)            218
(china, covid)           212
(china, day)             211
(china, national)        185
(report, case)           183
(new, case)              181
(national, day)          170
(china, country)         152
(covid, case)            152
(china, province)        151
(china, city)            149
(poverty, china)         127
(new, china)             125
(covid, say)             125
(new, report)            124
(people, china)          121
(china, world)           120
(day, holiday)           120
(test, positive)         117
(new, covid)             117
(president, china)       111
(china, xinjiang)        110
(china, holiday)         108
(covid, pandemic)        108
(china, development)     108
(first, china)           105
(global, china)          104
(say, president)         101
(president, trump)        97

USA:
                          weight
(president, trump)          1037
(say, covid)          

In [206]:
def get_graph(network):
  up_weighted = []
  for edge in network:
      #we can filter edges by weight by uncommenting the next line and setting desired weight threshold
      up_weighted.append((edge[0],edge[1],network[edge]))
      
  #print(network)
  #print(up_weighted[0:10])
  G = nx.Graph()
  G.add_weighted_edges_from(up_weighted)
  return G

In [207]:
G_China = get_graph(network_China)
G_USA = get_graph(network_USA)
G_China_USA = get_graph(network_China_USA)

In [208]:
print('China:')
print('Nodes: ',len(G_China.nodes()))
print('Edges: ',len(G_China.edges()))
print('Is connected: ',nx.is_connected(G_China))
print()
print('USA:')
print('Nodes: ',len(G_USA.nodes()))
print('Edges: ',len(G_USA.edges()))
print('Is connected: ',nx.is_connected(G_USA))
print()
print('China&USA:')
print('Nodes: ',len(G_China_USA.nodes()))
print('Edges: ',len(G_China_USA.edges()))
print('Is connected: ',nx.is_connected(G_China_USA))

China:
Nodes:  803
Edges:  53298
Is connected:  True

USA:
Nodes:  720
Edges:  68156
Is connected:  True

China&USA:
Nodes:  771
Edges:  90251
Is connected:  True


#Save edge list

In [209]:
filename = './edgelist_China_2_700.csv'
nx.write_weighted_edgelist(G_China, filename, delimiter=",")
#add header with appropriate column names (works on colab and Linux/Mac(?))
!sed -i.bak 1i"Source,Target,Weight" ./edgelist_China_2_700.csv
files.download("edgelist_China_2_700.csv")

filename = './edgelist_USA_2_700.csv'
nx.write_weighted_edgelist(G_USA, filename, delimiter=",")
#add header with appropriate column names (works on colab and Linux/Mac(?))
!sed -i.bak 1i"Source,Target,Weight" ./edgelist_USA_2_700.csv
files.download("edgelist_USA_2_700.csv")

filename = './edgelist_China_USA_2_700.csv'
nx.write_weighted_edgelist(G_China_USA, filename, delimiter=",")
#add header with appropriate column names (works on colab and Linux/Mac(?))
!sed -i.bak 1i"Source,Target,Weight" ./edgelist_China_USA_2_700.csv
files.download("edgelist_China_USA_2_700.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>





# Create Node List




In [125]:
def nodes(freq_dict, name):
  word_nodes = pd.DataFrame.from_dict(freq_dict,orient="index")
  word_nodes.reset_index(inplace=True)
  word_nodes["Label"] = word_nodes["index"]
  word_nodes.rename(columns={"index":"Id",0:"delete"},inplace=True)
  word_nodes = word_nodes.drop(columns=['delete'])
  nodelist = pd.DataFrame()
  nodelist = nodelist.append(word_nodes, ignore_index=True)

  nodelist = nodelist.to_csv("nodelist_"+name+".csv",index=False)
  files.download("nodelist_"+name+".csv")
  return nodelist, word_nodes

In [210]:
nodelist_China, word_nodes_China = nodes(freq_dict_China,"China_2_700")
nodelist_USA, word_nodes_USA = nodes(freq_dict_USA,"USA_2_700")
nodelist_China_USA, word_nodes_China_USA = nodes(freq_dict_China_USA,"China_2_USA_700")

print('China:')
print(word_nodes_China.head())
print()
print('USA:')
print(word_nodes_USA.head())
print()
print('China&USA:')
print(word_nodes_China_USA.head())
print()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

China:
      Id  Label
0  china  china
1  covid  covid
2    say    say
3   case   case
4    new    new

USA:
            Id        Label
0        covid        covid
1  coronavirus  coronavirus
2          say          say
3        trump        trump
4          new          new

China&USA:
            Id        Label
0        covid        covid
1        china        china
2  coronavirus  coronavirus
3          say          say
4          new          new

